In [144]:
from datetime import datetime, timedelta
# from calender import mdays
import pandas as pd
import psycopg2 as pg

import warnings

warnings.filterwarnings('ignore')

In [145]:
today = datetime.now().date()
today

datetime.date(2023, 3, 14)

In [146]:
engine = pg.connect("dbname='postgres' user='postgres' host='localhost' port='5432' password='lungmaker'")

In [147]:
query = "select * FROM public.package"
package = pd.read_sql_query(query,con=engine)

In [148]:
query = "select * FROM public.package_extra"
package_extra = pd.read_sql_query(query,con=engine)

In [149]:
engine.close()

In [150]:
def date_update_frequency(date_,frequency):
    if (date_.month+frequency) > 12:
        month_ = (date_.month + frequency) - 12
        year_ = date_.year +1

        date_update =  date_.replace(year=year_).replace(month=month_)
        return date_update
    else:
        month_ = (date_.month + frequency)
        date_update = date_.replace(month=month_)
        return date_update


In [151]:
def status(a):
    if a.days > 60:
        return ("ยังไม่ถึงเวลาที่กำหนด")

    elif a.days > 0 and  a.days < 60:
        return ("ใกล้ถึงเวลาที่กำหนด")

    elif a.days <= 0:
        return ("เกินเวลาที่กำหนด")

    else:
        pass
         

In [152]:
for idx,row in package.iterrows():
    update_frequency = package_extra.loc[
            package_extra['package_id'].str.contains(row['id']) & 
            package_extra['key'].str.contains("update_frequency_interval|update_frequency_unit")
            ].loc[package_extra['key'] != "update_frequency_unit_other"][['key','value']].set_index('key').T

    package.loc[idx,'update_frequency_unit'] = update_frequency['update_frequency_unit']['value']
    package.loc[idx,'update_frequency_interval'] = update_frequency['update_frequency_interval']['value']


    if update_frequency['update_frequency_unit']['value'] == 'ปี':
        date_next = row['metadata_modified'].date().replace(year=row['metadata_modified'].date().year+1)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color

    elif update_frequency['update_frequency_unit']['value'] == 'ครึ่งปี':
        date_next = date_update_frequency(row['metadata_modified'].date(),6)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color

    elif update_frequency['update_frequency_unit']['value'] == 'ไตรมาส':
        date_next = date_update_frequency(row['metadata_modified'].date(),3)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color
         
    elif update_frequency['update_frequency_unit']['value'] == 'เดือน':
        # print(row['metadata_modified'].date() + timedelta(mdays[row['metadata_modified'].date().month]))
        date_next = date_update_frequency(row['metadata_modified'].date(),1)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color

    elif update_frequency['update_frequency_unit']['value'] == 'สัปดาห์':
        date_next = row['metadata_modified'].date() + timedelta(weeks=1)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color
    
    elif update_frequency['update_frequency_unit']['value'] == 'วัน':
        date_next = row['metadata_modified'].date() + timedelta(days=1)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color

    elif update_frequency['update_frequency_unit']['value'] == 'ชั่วโมง':
        date_next = row['metadata_modified'].date() + timedelta(hours=1)
        days_left = date_next - today
        status_color = status(days_left)
        
        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color

    elif update_frequency['update_frequency_unit']['value'] == 'นาที':
        date_next = row['metadata_modified'].date() + timedelta(minutes=1)
        days_left = date_next - today
        status_color = status(days_left)

        package.loc[idx,'update_next'] = date_next
        package.loc[idx,'days_left'] = days_left
        package.loc[idx,'status_color'] = status_color

    else:
        pass
        # date_next = row['metadata_modified'].date()
        # days_left = date_next - today

        # package.loc[idx,'update_frequency_next'] = np.nan
        # package.loc[idx,'days_left'] = np.nan

In [153]:
package.tail(2)

,id,name,title,version,url,notes,author,author_email,maintainer,maintainer_email,...,owner_org,private,metadata_modified,creator_user_id,metadata_created,update_frequency_unit,update_frequency_interval,update_next,days_left,status_color
38,b3951945-4330-482a-8289-b28a5520587c,dataset-gdpublish-pap2,ทรัพยากรธรรมชาติและสิ่งแวดล้อม,None,https://data.go.th/dataset/gdpublish-pap2,ระบบกำจัดของเสียรวมขององค์กรปกครองส่วนท้องถิ่น...,None,None,กองยุทธศาสตร์และแผนงาน /กลุ่มงานวิเคราะห์ประสา...,strategy.sarabun@onep.go.th,...,ab0eab12-e0d9-49cf-a5ba-fb3c71bc6000,False,2023-02-10 06:42:47.521013,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 06:41:49.306128,ครึ่งปี,1,2023-08-10,149 days,ยังไม่ถึงเวลาที่กำหนด
39,d4ce66b2-aeb8-42ab-82cf-343b76f4c6ad,dataset-action-plan-2021,วิทยาศาสตร์,None,https://data.go.th/dataset/action-plan-2021,แผนปฏิบัติราชการงานวิเทศสัมพันธ์และกิจการอาเซี...,None,None,ดร. ชฎาพร โชติรดาภาณ์,chotirada@gmail.com,...,ab0eab12-e0d9-49cf-a5ba-fb3c71bc6000,False,2023-02-10 06:50:05.056362,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 06:43:08.909443,ครึ่งปี,2,2023-08-10,149 days,ยังไม่ถึงเวลาที่กำหนด


In [154]:
package['days_left'] = package['days_left'].astype(str).replace('NaT','')
package['update_next'] = pd.to_datetime(package['update_next']).astype(str).replace('NaT','')

In [155]:
print(date_next - today)
print(type(date_next - today))

149 days, 0:00:00
<class 'datetime.timedelta'>


In [156]:
package.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         40 non-null     object        
 1   name                       40 non-null     object        
 2   title                      40 non-null     object        
 3   version                    29 non-null     object        
 4   url                        40 non-null     object        
 5   notes                      40 non-null     object        
 6   author                     29 non-null     object        
 7   author_email               29 non-null     object        
 8   maintainer                 40 non-null     object        
 9   maintainer_email           40 non-null     object        
 10  state                      40 non-null     object        
 11  license_id                 40 non-null     object        
 12  type      

In [157]:
package.iloc[:,-8:]

,metadata_modified,creator_user_id,metadata_created,update_frequency_unit,update_frequency_interval,update_next,days_left,status_color
0,2023-02-10 04:43:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 04:41:00.000000,ตามเวลาจริง,เมื่อมีการปรับปรุงข้อมูล,,,NaN
1,2023-02-10 06:26:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 06:24:00.000000,เดือน,1,2023-03-10,-4 days,เกินเวลาที่กำหนด
2,2023-02-10 04:18:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 03:31:00.000000,ปี,1,2024-02-10,333 days,ยังไม่ถึงเวลาที่กำหนด
3,2023-02-10 04:57:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 04:55:00.000000,ไม่ทราบ,1,,,NaN
4,2023-02-09 17:00:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-01-31 03:07:00.000000,ไม่ทราบ,,,,NaN
5,2023-02-10 06:52:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-09 06:38:00.000000,ไตรมาส,1,2023-05-10,57 days,ใกล้ถึงเวลาที่กำหนด
6,2023-02-10 06:52:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 04:52:00.000000,ไม่มีการปรับปรุงหลังจากการจัดเก็บข้อมูล,1,,,NaN
7,2023-02-10 06:35:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 06:32:00.000000,เดือน,1,2023-03-10,-4 days,เกินเวลาที่กำหนด
8,2023-02-10 06:51:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 06:31:00.000000,ไตรมาส,1,2023-05-10,57 days,ใกล้ถึงเวลาที่กำหนด
9,2023-02-10 06:42:00.000000,b8f27ddf-8bec-4dd1-90e1-da95e5961386,2023-02-10 06:41:00.000000,ครึ่งปี,1,2023-08-10,149 days,ยังไม่ถึงเวลาที่กำหนด


In [158]:
package.to_csv('update_date.csv',index=False,encoding='utf-8-sig')